In [32]:
import numpy as np
np.set_printoptions(3, suppress=True)
import mujoco
from mim_data_utils import MujocoVisualizer, Scene, Logger

In [2]:
# Use a subprocess to do the actual writing.
log_writer = Logger.to_subprocess()

# Initialize the websocket server to send data to the visualizer.
log_writer.with_websocket()

In [44]:
logger = Logger(log_writer)
logger.clear()  # Calling clear to remove all previous logged data.
logger.layout('qpos[:3];qvel[0],qvel[1],qvel[2]')

# Loading mujoco scene.
mj_model = mujoco.MjModel.from_xml_path('scene.xml')
mj_data = mujoco.MjData(mj_model)
dt = 0.001

# Visualizer object.
mj_viz = MujocoVisualizer(mj_model, mj_data)

# The scene holds all 3d object data.
scene = Scene()

# Add the mujoco objects to the scene.
mj_viz.populate_scene(scene)

# Add color to the objects.
for name, entry in scene.entries().items():
    material = entry['object'].material
    
    if name == 'floor_0':
        material['color'] =  np.array([168, 82, 53])
    elif name == 'sphere_1':
         material['color'] = 'dddddddd'  # Transparent gray
    
# This logs the 3d data.
logger.log_static(scene)

# Set location of camera 0.
logger.camera_location(0, np.array([-0.2, -0.2, 0.3]), np.zeros(3))

# Setup initial position of sphere.
mj_data.qpos = [0, 0, 0.1, 1, 0, 0, 0]
mj_data.qvel = [1, 0, 0, 0, 0, 0]

# 
t = 0

# You need to step before calling update_scene to populate all the
# mujoco data.
mujoco.mj_step(mj_model, mj_data)
mj_viz.update_scene()
logger.log(scene, t)

In [45]:
mj_data.qpos = [0, 0, 0.1, 1, 0, 0, 0]
mj_data.qvel = [1, 0, 0, 0, 0, 0]

for ti in range(500):
    # Send 3d updates only every 5 ms. Logging the full scene updates
    # is a bit slow at the moment to do it for every dt.
    if ti % 5 == 0:
        mj_viz.update_scene()
        logger.log(scene, t)

    # Log some additional data.
    # Note how the 3d data is logged only every 5 ms while the 
    # timeseries for position and velocity is logged every 1 ms.
    # As long as the timeseries themself are monotonic increasing,
    # the individual timeseries can be logged at arbitrary times.
    logger.log({
        'qpos': mj_data.qpos,
        'qvel': mj_data.qvel
    }, t)

    # Calling this after every control cycle avoids high latency spikes.
    # Flushing often is much faster than flushing every few times.
    logger.flush()
    
    mujoco.mj_step(mj_model, mj_data)
    t += dt